In [ ]:
import random
import math
import cmath

In [ ]:
# Define the Node Structure
class Node:
    def __init__(self, type, value=None):
        self.type = type
        self.value = value
        self.children = []

    def evaluate(self, input_values, available_variables):
      if self.value is None:
          return None  # Return None for nodes with no value (such as non-terminal nodes)
      elif isinstance(self.value, str):
          if self.value == '+':
              return self.children[0].evaluate(input_values, available_variables) + self.children[1].evaluate(input_values, available_variables)
          elif self.value == '-':
              return self.children[0].evaluate(input_values, available_variables) - self.children[1].evaluate(input_values, available_variables)
          elif self.value == '*':
              return self.children[0].evaluate(input_values, available_variables) * self.children[1].evaluate(input_values, available_variables)
          elif self.value == '/':
              divisor = self.children[1].evaluate(input_values, available_variables)
              if divisor != 0:
                  return self.children[0].evaluate(input_values, available_variables) / divisor
              else:
                  return 0  # Return a default value for division by zero
          elif self.value == '^':
              base = self.children[0].evaluate(input_values, available_variables)
              exponent = self.children[1].evaluate(input_values, available_variables)
              if base != 0 or (exponent >= 0 and exponent.imag == 0):
                  return base ** exponent
              else:
                  return 0  # Return a default value for zero base raised to negative or complex exponent
          elif self.value == 'sin':
              return math.sin(self.children[0].evaluate(input_values, available_variables))
          elif self.value == 'cos':
              return math.cos(self.children[0].evaluate(input_values, available_variables))
          elif self.value in available_variables:
              return input_values[available_variables.index(self.value)]
      else:
          return float(self.value)

    def print_function(self):
        if self.value == '+':
            return f"({self.children[0].print_function()} + {self.children[1].print_function()})"
        elif self.value == '-':
            return f"({self.children[0].print_function()} - {self.children[1].print_function()})"
        elif self.value == '*':
            return f"{self.children[0].print_function()} * {self.children[1].print_function()}"
        elif self.value == '/':
            return f"{self.children[0].print_function()} / {self.children[1].print_function()}"
        elif self.value == '^':
            return f"{self.children[0].print_function()} ^ {self.children[1].print_function()}"
        elif self.value == 'sin':
            return f"sin({self.children[0].print_function()})"
        elif self.value == 'cos':
            return f"cos({self.children[0].print_function()})"
        elif self.value in available_variables:
            return self.value
        else:
            return str(float(self.value))

In [ ]:
# Generate Initial Population
def generate_tree(max_depth, available_operators, available_variables):
    if max_depth == 0 or random.random() < 0.1:
        if random.random() < 0.5:
            return Node('constant', value=random.uniform(-10, 10))
        else:
            return Node('variable', value=random.choice(available_variables))
    else:
        node = Node('operator', value=random.choice(available_operators))
        num_children = 2 if node.value not in ['sin', 'cos'] else 1
        for _ in range(num_children):
            node.children.append(generate_tree(max_depth - 1, available_operators, available_variables))
        return node

In [ ]:
# Define the Fitness Function
def fitness_function(tree, input_data, target_outputs, available_variables):
    total_error = 0.0
    for input_values, target_output in zip(input_data, target_outputs):
        output = tree.evaluate(input_values, available_variables)
        error = abs(output - target_output)
        total_error += error
    return total_error

In [ ]:
# Genetic Operators
def crossover(parent1, parent2):
    child = parent1
    if parent1.children and parent2.children:
        node1 = random.choice(parent1.children)
        node2 = random.choice(parent2.children)
        node1_index = parent1.children.index(node1)
        node2_index = parent2.children.index(node2)

        # Extend child's children list if needed
        if node1_index >= len(child.children):
            child.children.extend([None] * (node1_index - len(child.children) + 1))
        if node2_index >= len(child.children):
            child.children.extend([None] * (node2_index - len(child.children) + 1))

        child.children[node1_index] = node2
        child.children[node2_index] = node1
    return child

In [ ]:
def mutation(tree, max_depth, available_operators, available_variables):
    mutated_tree = tree
    if mutated_tree.children:
        node = random.choice(mutated_tree.children)
        node_index = mutated_tree.children.index(node)
        mutated_tree.children[node_index] = generate_tree(max_depth, available_operators, available_variables)
    return mutated_tree

In [ ]:
def create_random_individual(max_depth, available_operators, available_variables):
    if max_depth == 0 or random.random() < 0.3:
        # Create a terminal node
        if random.random() < 0.5:
            # Generate a random variable
            return Node(random.choice(available_variables))
        else:
            # Generate a random constant
            return Node(random.uniform(-10, 10))
    else:
        # Create an operator node
        operator = random.choice(available_operators)
        children = [
            create_random_individual(max_depth - 1, available_operators, available_variables),
            create_random_individual(max_depth - 1, available_operators, available_variables)
        ]
        return Node(operator, children)

In [ ]:
# Evolutionary Loop
def genetic_programming(input_data, target_outputs, available_operators, available_variables, population_size, max_depth, num_generations):
    population = []
    for _ in range(population_size):
        individual = create_random_individual(max_depth, available_operators, available_variables)
        population.append(individual)

    for generation in range(num_generations):
        fitness_scores = []
        for individual in population:
            fitness_scores.append(fitness_function(individual, input_data, target_outputs, available_variables))

        # Select parents for reproduction
        parents = select_parents(population, fitness_scores)

        # Create the next generation
        population = []
        for _ in range(population_size):
            parent1, parent2 = random.choices(parents, k=2)
            child = crossover(parent1, parent2)
            child = mutate(child, available_operators, available_variables)
            population.append(child)

    best_individual = min(population, key=lambda individual: fitness_function(individual, input_data, target_outputs, available_variables))
    return best_individual

In [ ]:
# Example Usage
input_data = [[1], [2], [3], [4], [5]]
target_outputs = [3, 5, 7, 9, 11]
available_operators = ['+', '-', '*', '/', '^', 'sin', 'cos']  # Available operators in the genetic program
available_variables = ['x', 'y', 'z']  # Available variables in the genetic program

population_size = 100
max_depth = 5
num_generations = 50

best_individual = genetic_programming(input_data, target_outputs, available_operators, available_variables, population_size, max_depth, num_generations)

# Print the best individual and its fitness score
print("Best Individual:", best_individual)
fitness_score = fitness_function(best_individual, input_data, target_outputs)
print("Best Individual:")
print(best_individual.print_function())
print("Fitness Score:", fitness_score)

TypeError: ignored

In [ ]:
# # Generate training data
# np.random.seed(1)
# X = np.random.uniform(-2*np.pi, 2*np.pi, 1000)  # Input features with 1000 samples
# y = np.sin(X)  # Target variable is the sine of X